# ESMAP Liberia dataset

In [1]:
import os
import sys
import pandas as pd

sys.path.append("../../")  # Adds higher directory to python modules path.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import numpy as np
import pandas as pd
from core.ODEDataset import ODEDataset
from utils import common_modifiers, dwelling_modifiers, socio_modifiers, finance_modifiers, appliances_modifiers, \
    geospatial, energy_modifiers, helpers
from utils import constants
import pathlib

In [2]:
DB_ROOT = "../../playground/data/ESMAP/Liberia_L/csv_data"
root = pathlib.Path(DB_ROOT)
ID_COL = "ID"
HHH_Position = 'HHH_position'
Survey_date = 2020

## Categories

In [3]:
Solar_device_original2final = {
    1: 'Solar home system',
    2: 'Solar lighting system',
    3: "Solar lanterns",
    4: np.nan
}

Connection_type_original2final = {
    1: 'National Grid Connection',
    2: 'Local Mini Grid connection',
    3: 'Electric generator',
    4: 'Solar Home System',
    5: 'Solar Lantern/Lighting System',
    6: 'Rechargeable Battery',
    8: 'Dry-cell battery',
    9: 'No electricity',
    555: 'Other'
}
#

Relation_to_HHH_original2final = {
    1: 'Head',
    2: 'Wife/Spouse',
    3: 'Child/adopted child',
    4: 'Grandchild',
    5: 'Niece/Nephew',
    6: 'Father/Mother',
    7: 'Sister/Brother',
    8: 'Son/Daughter-in-law',
    9: 'Brother/Sister-in-law',
    10: 'Father/Mother-in-law',
    11: 'Grandfather/mother',
    12: 'Other relative',
    13: 'Servant/servant’s relative',
    14: 'Other non-relative'
}
Main_occupation_original2final = {
    14: 'Unemployed',
    9: 'Unemployed',
    1: 'Employee non-farm',
    5: 'Own-account worker farm',
    7: 'Worker not classifiable by status',
    3: 'Own-account worker non-farm',
    13: 'Unemployed',
    2: 'Employee farm',
    11: 'Unemployed',
    4: 'Own-account worker non-farm',
    10: 'Unemployed',
    6: 'Contributing family worker',
    555: 'Worker not classifiable by status',
    12: 'Unemployed',
    8: 'Worker not classifiable by status',
    19: 'Unemployed',
    17: 'Unemployed',
    16: 'Unemployed',
    15: 'Unemployed',
    18: 'Unemployed',
    20: 'Unemployed',
    21: 'Unemployed'
}

Education_level_original2final = {
    1: 'No schooling',
    2: 'Primary education',
    3: 'Lower secondary education',
    4: 'Post-secondary non-tertiary education',
    5: 'Post-secondary non-tertiary education',
    6: "Doctoral or equivalent level",
    7: np.nan
}

Dwelling_wall_Quality_original2final = {
    1: 'Wood and mud',
    2: 'Wood and thatch',
    3: 'Wood only',
    4: 'Stone only',
    5: 'Stone and mud',
    6: 'Stone and cement',
    7: 'Blocks, plastered with cement',
    8: 'Blocks, unplastered',
    9: 'Bricks',
    10: 'Mud bricks (traditional)',
    11: 'Steel',
    12: 'Cargo container',
    13: 'Parquet or polished wood',
    14: 'Chip wood',
    15: 'Corrugated iron/zinc sheet',
    16: 'Asbestos',
    17: 'Reed/bamboo',
    555: 'Other'
}

Dwelling_roof_Quality_original2final = {
    1: 'Wood and mud',
    2: 'Wood and thatch',
    3: 'Stone and Cement',
    4: 'Bricks',
    5: 'Corrugated iron sheet',
    6: 'Asbestos',
    7: 'Reed/bamboo',
    8: 'Plastic canvas/Tarpaulin',
    555: 'Other'
}

Dwelling_floor_Quality_original2final = {
    1: 'Mud/Dung',
    2: 'Reed/bamboo',
    3: 'Wood planks',
    4: 'Parquet or polished wood',
    5: 'Cement screed',
    6: 'Plastic tiles',
    7: 'Cement tiles',
    8: 'Brick tiles',
    9: 'Ceramic/Marble tiles',
    555: 'Other'
}

# Dwelling_toilet_quality_final2original = {
#     1: 'None (open field)',
#     2: 'Flush to sewage',
#     3: 'Flush to septic tank',
#     4: 'Pail/Bucket',
#     5: 'Covered pit latrine',
#     6: 'Uncovered pit latrine',
#     7: 'Community latrine',
#     555: 'Other'
# }
Dwelling_toilet_quality_final2original = {
    1: 1,
    2: 2,
    3: 3,
    4: 4,
    5: 5,
    6: 6,
    7: 7,
    555: 0
}
Dwelling_water_quality_final2original = {
    1: 'Pipe borne water treated',
    2: 'Pipe borne water untreated',
    3: 'Bore hole/ hand pump',
    4: 'Electric water pump',
    5: 'Well/spring protected',
    6: 'Well/spring unprotected',
    7: 'River/spring',
    8: 'Lake/reservoir',
    9: 'Rain water',
    10: 'Tanker/truck/vendor',
    11: 'Sachet water',
    12: 'Bottle water',
    555: 'Other'
}

Ownership_motorized_vehicle_original2final = {
    'Vehicle (Car, pickup truck, etc)': 1,
    'Bicycle': 0,
    'Domestic water pump': 0,
    'Motorcycle': 1,
    'Other equipment': 0,
    'Tractor': 0
}


## Clusters

In [4]:
Weekly_Expenditures = [
    'L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11'
]

Monthly_Expenditures = ['_30daysL12', '_30daysL13', '_30daysL14', '_30daysL15', '_30daysL16', '_30daysL17',
                        '_30daysL18', '_30daysL19']

Yearly_Expenditures = ['_1yearL20', '_1yearL21', '_1yearL22', '_1yearL23', '_1yearL24', '_1yearL25', '_1yearL26',
                       '_1yearL27', '_1yearL28', '_1yearL29', '_1yearL30', '_1yearL31', '_1yearL32', '_1yearL33',
                       '_1yearL34']

## MTF_HH_Characteristics
specify the characteristics of the interviewee and the household

In [5]:
MTF_HH_Characteristics = ODEDataset("liberia/MTF_HH_Characteristics")
MTF_HH_Characteristics.from_csv(root.joinpath("MTF_HH_Characteristics.csv"))

MTF_HH_Characteristics = MTF_HH_Characteristics.drop_columns([
    'SETOFA', 'KEY'
])

MTF_HH_Characteristics = MTF_HH_Characteristics.group_by('PARENT_KEY')
MTF_HH_Characteristics = MTF_HH_Characteristics.apply(common_modifiers.rename({
    'PARENT_KEY': ID_COL
}))

rename some columns and make them more readable

In [6]:

MTF_HH_Characteristics = MTF_HH_Characteristics.apply(common_modifiers.rename({
    "A4": 'Relation_HHH',
    "A5": 'Age',
    'A7': 'Education_exists',
    'A8': 'Is_student',
    'A9': 'Highest_education',
    'A9A': 'Years_of_education',
    'A14': "Main_occupation_raw",
}))

MTF_HH_Characteristics = MTF_HH_Characteristics.apply(common_modifiers.transform_list_int('Highest_education'))

MTF_HH_Characteristics = MTF_HH_Characteristics.apply(common_modifiers.add_const_driver_many({
    'Survey_date': 2020
}))

MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature(HHH_Position,
                                                            socio_modifiers.extract_household_position('Relation_HHH',
                                                                                                       1))


In [7]:
MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('Highest_eduction_HHH_raw',
                                                            common_modifiers.extract_feature_by_position(
                                                                'Highest_education', HHH_Position))

MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature("Education_level_HHH",
                                                            common_modifiers.categorize('Highest_eduction_HHH_raw',
                                                                                        Education_level_original2final))
MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('Age_HHH',
                                                            common_modifiers.extract_feature_by_position(
                                                                'Age', HHH_Position))

MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('Number_adults',
                                                            socio_modifiers.extract_age_groups('Age', 'adults'))

# Socio_status_HHH
MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('Socio_status_HHH_raw',
                                                            common_modifiers.extract_feature_by_position(
                                                                'Main_occupation_raw', HHH_Position))

MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('Socio_status_HHH',
                                                            common_modifiers.categorize('Socio_status_HHH_raw',
                                                                                        Main_occupation_original2final))

MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('Main_occupation',
                                                            lambda x: x['Main_occupation_raw'][0])


def get_hh_with_home_business(row):
    if helpers.is_nan(row['Socio_status_HHH']):
        return 'No'
    if row['Socio_status_HHH'] != 'Unemployed':
        return 'Yes'
    return 'No'


MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('HH_with_home_business', get_hh_with_home_business)

MTF_HH_Characteristics = MTF_HH_Characteristics.new_feature('sample_size', lambda x: len(x['Relation_HHH']))



In [8]:
MTF_HH_Characteristics = MTF_HH_Characteristics.select([
    ID_COL, 'Age_HHH', 'Education_level_HHH', 'Socio_status_HHH', 'HH_with_home_business', 'Number_adults',
    'sample_size'
])

In [9]:
MTF_HH_Characteristics.preview()

,ID,Age_HHH,Education_level_HHH,Socio_status_HHH,HH_with_home_business,Number_adults,sample_size
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,55.0,Doctoral or equivalent level,Unemployed,No,6.0,9
1,uuid:00096f15-d833-48e1-9f88-e560ce19eee0,42.0,Lower secondary education,Unemployed,No,2.0,5
2,uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2,39.0,NaN,Unemployed,No,2.0,5
3,uuid:002662e7-c92b-4c89-9a80-73c90a86face,51.0,NaN,Employee non-farm,Yes,3.0,3
4,uuid:0036e1d5-595d-4860-90eb-83f17be07002,37.0,NaN,Employee non-farm,Yes,2.0,5


## MTF_HH_Main

In [10]:
MTF_HH_Main = ODEDataset("liberia/MTF_HH_Main")
MTF_HH_Main.from_csv(root.joinpath("MTF_HH_Main_gadm_revised.csv"))

MTF_HH_Main = MTF_HH_Main.group_by('KEY')
MTF_HH_Main = MTF_HH_Main.apply(common_modifiers.rename({
    "KEY": ID_COL
}))


In [11]:
MTF_HH_Main = MTF_HH_Main.apply(common_modifiers.rename({
    'B8': 'Number_of_rooms'
}))

MTF_HH_Main = MTF_HH_Main.apply(common_modifiers.take('Number_of_rooms', 0))


### Dwelling_quality_index


| Driver | Variable  | Question                                                        | Source          |
|--------|-----------|-----------------------------------------------------------------|-----------------|
| Walls  | B9        | The walls of the dwelling are mainly made of what material?     | HH_Roaster_Main |
| Walls  | B9_other  | The walls of the dwelling are mainly made of what material?     | HH_Roaster_Main |
| Roof   | B10       | The roof of the dwelling is mainly made of what material? (...  | HH_Roaster_Main |
| Floor  | B10_other | The floor of the dwelling is mainly made of what material?      | HH_Roaster_Main |
| Floor  | B11       | The floor of the dwelling is mainly made of what material?      | HH_Roaster_Main |
| Toilet | B11_other | What type of toilet facility does your household use? (#1/8)    | HH_Roaster_Main |
| Toilet | B12_1     | What type of toilet facility does your household use? (#2/8)    | HH_Roaster_Main |
| Toilet | B12_2     | What type of toilet facility does your household use? (Other)   | HH_Roaster_Main |
| Water  | B12_other | What is your household’s main source of drinking water?         | HH_Roaster_Main |
| Water  | B13       | What is your household’s main source of drinking water? (Ot...) | HH_Roaster_Main |
| Water  | B13_other | Is your drinking water treated (chemical treatment)?            | HH_Roaster_Main |


In [12]:
MTF_HH_Main = MTF_HH_Main.new_feature('Dwelling_wall', lambda x: x['B9'])
MTF_HH_Main = MTF_HH_Main.new_feature('Dwelling_roof', lambda x: x['B10'])
MTF_HH_Main = MTF_HH_Main.new_feature('Dwelling_floor', lambda x: x['B11'])
MTF_HH_Main = MTF_HH_Main.new_feature('Dwelling_water', lambda x: x['B13'])
MTF_HH_Main = MTF_HH_Main.new_feature('Dwelling_toilet', lambda x: x['B12_1'])
MTF_HH_Main = MTF_HH_Main.new_feature('Dwelling_quality_index', dwelling_modifiers.dwelling_quality_index())

### Measurement_age


> this is The age of the connection if available, otherwise NAN.

> Rename some of the variables to make them more readable.

**solar devices:**

- Solar home system - > 1
- Solar lighting system -> 2
- Solar lanterns -> 3
- None -> 4

| New Name                               | Old Variable | Question                                                                 | Source          |
|----------------------------------------|--------------|--------------------------------------------------------------------------|-----------------|
| Years_with_grid                        | C7           | How many years has your household been connected to the grid?            | HH_Roaster_Main |
| Is_household_connected_grid            | C2           | Is your household connected to the grid?                                 | HH_Roaster_Main |
| Is_household_connected_local_mini_grid | C38          | Is your household connected to the local mini grid?                      | HH_Roaster_Main |
| Years_with_local_mini_grid             | C41          | How many years has your household been connected to the local mini grid? | HH_Roaster_Main |
| solar_device                           | C122         | Do you have solar device ?                                               | HH_Roaster_Main |
| year_of_getting_solar_device           | C155         | In what year did you get your solar device                               | HH_Roaster_Main |

> We need to calculate the years of the solar device: Survey year - year_of_getting_solar_device

Then we extract the measurement age =>  the connection age to the energy source.


In [13]:

MTF_HH_Main = MTF_HH_Main.new_feature('Connected_to_grid', common_modifiers.is_in_list('C7', [1], 'Yes', 'No'))
MTF_HH_Main = MTF_HH_Main.new_feature('Connected_to_local_mini_grid',
                                      common_modifiers.is_in_list('C38', [1], 'Yes', 'No'))
MTF_HH_Main = MTF_HH_Main.new_feature('Has_solar_device', common_modifiers.is_in_list('C122_1', [1, 2, 3], 'Yes', 'No'))

MTF_HH_Main = MTF_HH_Main.new_feature('Connection_type', energy_modifiers.get_connection_type(
    'Connected_to_grid',
    'Connected_to_local_mini_grid',
    'Has_solar_device',
    {
        constants.NATIONAL_GRID: 'Yes',
        constants.LOCAL_MINI_GRID: 'Yes',
        constants.SOLAR_HOME_SYSTEM: 'Yes'
    }))


def calculate_solar_device_age(row):
    if row['Has_solar_device'] == 'Yes':
        solar_date = row['C155'][0]
        if helpers.is_nan(solar_date):
            return np.nan
        year = solar_date.split('-')[0]
        return Survey_date - int(year)
    return np.nan


MTF_HH_Main = MTF_HH_Main.new_feature('Solar_device_age', calculate_solar_device_age)


In [14]:
MTF_HH_Main.to_dataframe()[
    ['Connection_type', 'Connected_to_grid', 'C7', 'Connected_to_local_mini_grid', 'C41', 'Has_solar_device',
     'Solar_device_age'
     ]].head(200)

,Connection_type,Connected_to_grid,C7,Connected_to_local_mini_grid,C41,Has_solar_device,Solar_device_age
0,Local mini-grid,No,[nan],Yes,[1.0],No,NaN
1,None,No,[nan],No,[nan],No,NaN
2,None,No,[nan],No,[nan],No,NaN
3,None,No,[nan],No,[nan],No,NaN
4,None,No,[nan],No,[nan],No,NaN
...,...,...,...,...,...,...,...
195,None,No,[nan],No,[nan],No,NaN
196,None,No,[nan],No,[nan],No,NaN
197,None,No,[nan],No,[nan],No,NaN
198,National grid,Yes,[1.0],Yes,[1.0],Yes,3.0


In [15]:
def calculate_measurement_age(row):
    result = np.nan
    connection_type = row['Connection_type']
    if helpers.is_nan(connection_type):
        return result

    if connection_type == constants.NATIONAL_GRID:
        result = row['C7'][0]

    if connection_type == constants.LOCAL_MINI_GRID:
        result = row['C41'][0]

    if connection_type == constants.SOLAR_HOME_SYSTEM:
        result = row['Solar_device_age']

    return result

In [16]:
MTF_HH_Main = MTF_HH_Main.new_feature('Measurement_age', calculate_measurement_age)



### Monthly_expenditure


In [17]:
MTF_HH_Main = MTF_HH_Main.new_feature('Monthly_expenditure', finance_modifiers.expenditure_multi_section(
    Yearly_Expenditures, Monthly_Expenditures, Weekly_Expenditures, []
))



### GeoSpatial Data

In [18]:
gadm_level_1_df = pd.read_csv(root.joinpath('GADM_level_1.csv'))
gadm_level_2_df = pd.read_csv(root.joinpath('GADM_level_2.csv'))
gadm_level_3_df = pd.read_csv(root.joinpath('GADM_level_3.csv'))



In [19]:
MTF_HH_Main = MTF_HH_Main.apply(common_modifiers.add_const_driver("GADM_level_0", "Liberia"))

In [20]:
MTF_HH_Main.preview()

,ID,HH_ID,Date_of_questionnaire_submission,County,District,EA,Locality,Language,Household_structure,Number_of_apartment_units,...,Dwelling_toilet,Dwelling_quality_index,Connected_to_grid,Connected_to_local_mini_grid,Has_solar_device,Connection_type,Solar_device_age,Measurement_age,Monthly_expenditure,GADM_level_0
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,[339.0],[2017-03-16 18:28:00],[Lofa],[Quardu Gboni],[David Toe],[1.0],[1.0],[120.0],[1.0],...,[3.0],7.0,No,Yes,No,Local mini-grid,NaN,1.0,201120.000000,Liberia
1,uuid:00096f15-d833-48e1-9f88-e560ce19eee0,[5.0],[2017-05-27 13:24:00],[Maryland],[Pleebo/Sodeken],[Upper Garwula],[1.0],[2.0],[120.0],[1.0],...,[3.0],5.0,No,No,No,None,NaN,NaN,19025.000000,Liberia
2,uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2,[20.0],[2017-05-29 19:52:00],[Maryland],[Pleebo/Sodeken],[Upper Kpaquelleh],[1.0],[1.0],[120.0],[5.0],...,[3.0],5.0,No,No,No,None,NaN,NaN,18705.833333,Liberia
3,uuid:002662e7-c92b-4c89-9a80-73c90a86face,[252.0],[2017-04-25 05:07:00],[Nimba],[Webbo],[Goah],[2.0],[2.0],[440.0],[1.0],...,[5.0],3.0,No,No,No,None,NaN,NaN,7292.500000,Liberia
4,uuid:0036e1d5-595d-4860-90eb-83f17be07002,[22.0],[2017-05-17 09:03:00],[Gbapolu],[District # 3],[Seimavolah],[2.0],[1.0],[250.0],[3.0],...,[8.0],6.2,No,No,No,None,NaN,NaN,48839.500000,Liberia


In [21]:
MTF_HH_Main = MTF_HH_Main.new_feature("GADM_level_raw", geospatial.get_gadm(
    gadm_level_1_df,
    gadm_level_2_df,
    gadm_level_3_df,
    'County', 'District', 'EA', '', 'Liberia'
))

MTF_HH_Main = MTF_HH_Main.new_feature("GADM_level_1", lambda x: x['GADM_level_raw'][1])
MTF_HH_Main = MTF_HH_Main.new_feature("GADM_level_2", lambda x: x['GADM_level_raw'][2])
MTF_HH_Main = MTF_HH_Main.new_feature("GADM_level_3", lambda x: x['GADM_level_raw'][3])

MTF_HH_Main.preview()

,ID,HH_ID,Date_of_questionnaire_submission,County,District,EA,Locality,Language,Household_structure,Number_of_apartment_units,...,Has_solar_device,Connection_type,Solar_device_age,Measurement_age,Monthly_expenditure,GADM_level_0,GADM_level_raw,GADM_level_1,GADM_level_2,GADM_level_3
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,[339.0],[2017-03-16 18:28:00],[Lofa],[Quardu Gboni],[David Toe],[1.0],[1.0],[120.0],[1.0],...,No,Local mini-grid,NaN,1.0,201120.000000,Liberia,"[LBR.8_1, Lofa, nan, nan]",Lofa,NaN,NaN
1,uuid:00096f15-d833-48e1-9f88-e560ce19eee0,[5.0],[2017-05-27 13:24:00],[Maryland],[Pleebo/Sodeken],[Upper Garwula],[1.0],[2.0],[120.0],[1.0],...,No,None,NaN,NaN,19025.000000,Liberia,"[LBR.10.2_1, Maryland, Pleebo/Sodeken, nan]",Maryland,Pleebo/Sodeken,NaN
2,uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2,[20.0],[2017-05-29 19:52:00],[Maryland],[Pleebo/Sodeken],[Upper Kpaquelleh],[1.0],[1.0],[120.0],[5.0],...,No,None,NaN,NaN,18705.833333,Liberia,"[LBR.10.2_1, Maryland, Pleebo/Sodeken, nan]",Maryland,Pleebo/Sodeken,NaN
3,uuid:002662e7-c92b-4c89-9a80-73c90a86face,[252.0],[2017-04-25 05:07:00],[Nimba],[Webbo],[Goah],[2.0],[2.0],[440.0],[1.0],...,No,None,NaN,NaN,7292.500000,Liberia,"[LBR.12_1, Nimba, nan, nan]",Nimba,NaN,NaN
4,uuid:0036e1d5-595d-4860-90eb-83f17be07002,[22.0],[2017-05-17 09:03:00],[Gbapolu],[District # 3],[Seimavolah],[2.0],[1.0],[250.0],[3.0],...,No,None,NaN,NaN,48839.500000,Liberia,"[LBR.3_1, Gbapolu, nan, nan]",Gbapolu,NaN,NaN


In [22]:

MTF_HH_Main = MTF_HH_Main.new_feature("Climate_zone_lev_1",
                                      geospatial.gis_info_by_gadm_level('Climate_majority',
                                                                        gadm_level_1_df,
                                                                        'GADM_level_1',
                                                                        ))

MTF_HH_Main = MTF_HH_Main.new_feature("Climate_zone_lev_2",
                                      geospatial.gis_info_by_gadm_level('Climate_majority',
                                                                        gadm_level_2_df,
                                                                        'GADM_level_2',
                                                                        ))

MTF_HH_Main.preview()

,ID,HH_ID,Date_of_questionnaire_submission,County,District,EA,Locality,Language,Household_structure,Number_of_apartment_units,...,Solar_device_age,Measurement_age,Monthly_expenditure,GADM_level_0,GADM_level_raw,GADM_level_1,GADM_level_2,GADM_level_3,Climate_zone_lev_1,Climate_zone_lev_2
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,[339.0],[2017-03-16 18:28:00],[Lofa],[Quardu Gboni],[David Toe],[1.0],[1.0],[120.0],[1.0],...,NaN,1.0,201120.000000,Liberia,"[LBR.8_1, Lofa, nan, nan]",Lofa,NaN,NaN,11.739750,NaN
1,uuid:00096f15-d833-48e1-9f88-e560ce19eee0,[5.0],[2017-05-27 13:24:00],[Maryland],[Pleebo/Sodeken],[Upper Garwula],[1.0],[2.0],[120.0],[1.0],...,NaN,NaN,19025.000000,Liberia,"[LBR.10.2_1, Maryland, Pleebo/Sodeken, nan]",Maryland,Pleebo/Sodeken,NaN,19.993023,19.993023
2,uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2,[20.0],[2017-05-29 19:52:00],[Maryland],[Pleebo/Sodeken],[Upper Kpaquelleh],[1.0],[1.0],[120.0],[5.0],...,NaN,NaN,18705.833333,Liberia,"[LBR.10.2_1, Maryland, Pleebo/Sodeken, nan]",Maryland,Pleebo/Sodeken,NaN,19.993023,19.993023
3,uuid:002662e7-c92b-4c89-9a80-73c90a86face,[252.0],[2017-04-25 05:07:00],[Nimba],[Webbo],[Goah],[2.0],[2.0],[440.0],[1.0],...,NaN,NaN,7292.500000,Liberia,"[LBR.12_1, Nimba, nan, nan]",Nimba,NaN,NaN,13.762500,NaN
4,uuid:0036e1d5-595d-4860-90eb-83f17be07002,[22.0],[2017-05-17 09:03:00],[Gbapolu],[District # 3],[Seimavolah],[2.0],[1.0],[250.0],[3.0],...,NaN,NaN,48839.500000,Liberia,"[LBR.3_1, Gbapolu, nan, nan]",Gbapolu,NaN,NaN,16.407250,NaN


In [23]:

MTF_HH_Main = MTF_HH_Main.select([
    ID_COL, 'Dwelling_quality_index', 'Measurement_age', 'Monthly_expenditure', 'Number_of_rooms',
    'Climate_zone_lev_1', 'Climate_zone_lev_2'
])

## MTF_HH_SectionH

Only `Clean fuel`  is extracted from this dataset

In [24]:
MTF_HH_SectionH = ODEDataset("liberia/MTF_HH_SectionH")
MTF_HH_SectionH.from_csv(root.joinpath("MTF_HH_SectionH.csv"))
MTF_HH_SectionH = MTF_HH_SectionH.apply(common_modifiers.rename({
    "PARENT_KEY": ID_COL
}))

MTF_HH_SectionH = MTF_HH_SectionH.group_by(ID_COL)

MTF_HH_SectionH.preview()

,ID,H2_pos,H2_lbl,H2_rpt_pos,H4_9_1,H4_9_2,H4_9_3,H4_9_4,H4_9_5,H4_9_6,...,H11_12,H11_13,H11_14,H11b,H11c,H12,H13,H14,KEY,SETOFH2_rpt
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,[d],[Charcoal],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[nan],[nan],[2.0],[Sack - large (charcoal)],[5.0],[5.0],[1750.0],[uuid:0006bfdb-a021-44c9-8250-016c2fb79147/Ss-...,[uuid:0006bfdb-a021-44c9-8250-016c2fb79147/Ss-...
1,uuid:00096f15-d833-48e1-9f88-e560ce19eee0,[c],[Wood collected],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[nan],[nan],[7.0],[Large bundle (wood)],[20.0],[nan],[nan],[uuid:00096f15-d833-48e1-9f88-e560ce19eee0/Ss-...,[uuid:00096f15-d833-48e1-9f88-e560ce19eee0/Ss-...
2,uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2,[d],[Charcoal],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[nan],[nan],[1.0],[Kg],[4.0],[4.0],[1000.0],[uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2/Ss-...,[uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2/Ss-...
3,uuid:002662e7-c92b-4c89-9a80-73c90a86face,[c],[Wood collected],[1],[2.0],[nan],[nan],[nan],[nan],[nan],...,[nan],[nan],[nan],[7.0],[Large bundle (wood)],[18.0],[nan],[nan],[uuid:002662e7-c92b-4c89-9a80-73c90a86face/Ss-...,[uuid:002662e7-c92b-4c89-9a80-73c90a86face/Ss-...
4,uuid:0036e1d5-595d-4860-90eb-83f17be07002,[c],[Wood collected],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[nan],[nan],[6.0],[Small bundle (wood)],[12.0],[nan],[nan],[uuid:0036e1d5-595d-4860-90eb-83f17be07002/Ss-...,[uuid:0036e1d5-595d-4860-90eb-83f17be07002/Ss-...


In [25]:
MTF_HH_SectionH = MTF_HH_SectionH.new_feature('Fuel_usage', lambda x: x['H2_lbl'][0])
clean_fuels = [
    'LPG/cooking gas', 'Solar', 'Electric',
]
MTF_HH_SectionH = MTF_HH_SectionH.new_feature('Clean_fuel', energy_modifiers.is_clean_fuel(clean_fuels, 'Fuel_usage'))

MTF_HH_SectionH.preview(2000)

,ID,H2_pos,H2_lbl,H2_rpt_pos,H4_9_1,H4_9_2,H4_9_3,H4_9_4,H4_9_5,H4_9_6,...,H11_14,H11b,H11c,H12,H13,H14,KEY,SETOFH2_rpt,Fuel_usage,Clean_fuel
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,[d],[Charcoal],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[2.0],[Sack - large (charcoal)],[5.0],[5.0],[1750.0],[uuid:0006bfdb-a021-44c9-8250-016c2fb79147/Ss-...,[uuid:0006bfdb-a021-44c9-8250-016c2fb79147/Ss-...,Charcoal,No
1,uuid:00096f15-d833-48e1-9f88-e560ce19eee0,[c],[Wood collected],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[7.0],[Large bundle (wood)],[20.0],[nan],[nan],[uuid:00096f15-d833-48e1-9f88-e560ce19eee0/Ss-...,[uuid:00096f15-d833-48e1-9f88-e560ce19eee0/Ss-...,Wood collected,No
2,uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2,[d],[Charcoal],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[1.0],[Kg],[4.0],[4.0],[1000.0],[uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2/Ss-...,[uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2/Ss-...,Charcoal,No
3,uuid:002662e7-c92b-4c89-9a80-73c90a86face,[c],[Wood collected],[1],[2.0],[nan],[nan],[nan],[nan],[nan],...,[nan],[7.0],[Large bundle (wood)],[18.0],[nan],[nan],[uuid:002662e7-c92b-4c89-9a80-73c90a86face/Ss-...,[uuid:002662e7-c92b-4c89-9a80-73c90a86face/Ss-...,Wood collected,No
4,uuid:0036e1d5-595d-4860-90eb-83f17be07002,[c],[Wood collected],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[6.0],[Small bundle (wood)],[12.0],[nan],[nan],[uuid:0036e1d5-595d-4860-90eb-83f17be07002/Ss-...,[uuid:0036e1d5-595d-4860-90eb-83f17be07002/Ss-...,Wood collected,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,uuid:8640d259-1f25-4c8a-88f3-1362bc874169,[d],[Charcoal],[1],[2.0],[nan],[nan],[nan],[nan],[nan],...,[nan],[1.0],[Kg],[1.0],[1.0],[650.0],[uuid:8640d259-1f25-4c8a-88f3-1362bc874169/Ss-...,[uuid:8640d259-1f25-4c8a-88f3-1362bc874169/Ss-...,Charcoal,No
1996,uuid:86426eda-b6dd-4623-b89c-e118e5b60a12,[d],[Charcoal],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[2.0],[Sack - large (charcoal)],[3.0],[3.0],[600.0],[uuid:86426eda-b6dd-4623-b89c-e118e5b60a12/Ss-...,[uuid:86426eda-b6dd-4623-b89c-e118e5b60a12/Ss-...,Charcoal,No
1997,uuid:864357ea-f75b-4917-8562-4accd3d6dcaf,[d],[Charcoal],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[3.0],[Big plastic bag (charcoal)],[1.0],[1.0],[350.0],[uuid:864357ea-f75b-4917-8562-4accd3d6dcaf/Ss-...,[uuid:864357ea-f75b-4917-8562-4accd3d6dcaf/Ss-...,Charcoal,No
1998,uuid:86512740-2d76-4954-9a24-0d77342f22da,[d],[Charcoal],[1],[2.0],[4.0],[nan],[nan],[nan],[nan],...,[nan],[4.0],[Small plastic bag (Fill to the brim) (charcoal)],[90.0],[90.0],[900.0],[uuid:86512740-2d76-4954-9a24-0d77342f22da/Ss-...,[uuid:86512740-2d76-4954-9a24-0d77342f22da/Ss-...,Charcoal,No


In [26]:
MTF_HH_SectionH = MTF_HH_SectionH.select([
    ID_COL, 'Clean_fuel'
])

MTF_HH_SectionH.preview()

,ID,Clean_fuel
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,No
1,uuid:00096f15-d833-48e1-9f88-e560ce19eee0,No
2,uuid:0025fd97-17a1-4f20-b5bd-2bfbf573bde2,No
3,uuid:002662e7-c92b-4c89-9a80-73c90a86face,No
4,uuid:0036e1d5-595d-4860-90eb-83f17be07002,No


## MTF_HH_SectionM
only `Ownership_motorized_vehicle` is extracted from this dataset

In [27]:
MTF_HH_SectionM = ODEDataset("liberia/MTF_HH_SectionM")
MTF_HH_SectionM.from_csv(root.joinpath("MTF_HH_SectionM.csv"))
MTF_HH_SectionM = MTF_HH_SectionM.apply(common_modifiers.rename({
    "PARENT_KEY": ID_COL
}))

MTF_HH_SectionM = MTF_HH_SectionM.group_by(ID_COL)

MTF_HH_SectionM.preview()

,ID,M_pos,M_lbl,M_rpt_pos,Ma,Mb,Mc,Mc_other,Md1,Md2,Md3,Md4,Md5,Md6,Md7,Md8,Me,Mf,KEY,SETOFM_rpt
0,uuid:0158227d-b2fb-4661-a791-1a4db82abaca,[1.0],"[Vehicle (Car, pickup truck, etc)]",[1],[1.0],[888.0],[1.0],[nan],[1.0],[2.0],[3.0],[4.0],[5.0],[nan],[nan],[nan],[6.0],[5.0],[uuid:0158227d-b2fb-4661-a791-1a4db82abaca/Ss-...,[uuid:0158227d-b2fb-4661-a791-1a4db82abaca/Ss-...
1,uuid:052b5bb0-6fa0-4500-98bc-d75bf23adc03,[2.0],[Motorcycle],[1],[1.0],[12.0],[1.0],[nan],[1.0],[2.0],[3.0],[4.0],[nan],[nan],[nan],[nan],[18.0],[18.0],[uuid:052b5bb0-6fa0-4500-98bc-d75bf23adc03/Ss-...,[uuid:052b5bb0-6fa0-4500-98bc-d75bf23adc03/Ss-...
2,uuid:05c3a6f6-2b1b-4ee0-b0e8-fe2393440f2c,[2.0],[Motorcycle],[1],[1.0],[9.0],[1.0],[nan],[1.0],[8.0],[nan],[nan],[nan],[nan],[nan],[nan],[6.0],[6.0],[uuid:05c3a6f6-2b1b-4ee0-b0e8-fe2393440f2c/Ss-...,[uuid:05c3a6f6-2b1b-4ee0-b0e8-fe2393440f2c/Ss-...
3,uuid:06938c8c-019d-42be-a04f-edbbad315098,[2.0],[Motorcycle],[1],[1.0],[100.0],[1.0],[nan],[2.0],[3.0],[4.0],[5.0],[nan],[nan],[nan],[nan],[12.0],[12.0],[uuid:06938c8c-019d-42be-a04f-edbbad315098/Ss-...,[uuid:06938c8c-019d-42be-a04f-edbbad315098/Ss-...
4,uuid:0992c731-d1fb-45f2-abd2-fb3c620790f8,[2.0],[Motorcycle],[1],[1.0],[9.0],[1.0],[nan],[2.0],[3.0],[4.0],[nan],[nan],[nan],[nan],[nan],[3.0],[3.0],[uuid:0992c731-d1fb-45f2-abd2-fb3c620790f8/Ss-...,[uuid:0992c731-d1fb-45f2-abd2-fb3c620790f8/Ss-...


In [28]:
MTF_HH_SectionM = MTF_HH_SectionM.new_feature('Ownership_motorized_vehicle',
                                              common_modifiers.categorize('M_lbl',
                                                                          Ownership_motorized_vehicle_original2final)).apply(
    common_modifiers.take('Ownership_motorized_vehicle', 0))

MTF_HH_SectionM = MTF_HH_SectionM.select([
    ID_COL, 'Ownership_motorized_vehicle'
])

MTF_HH_SectionM.preview()

,ID,Ownership_motorized_vehicle
0,uuid:0158227d-b2fb-4661-a791-1a4db82abaca,1.0
1,uuid:052b5bb0-6fa0-4500-98bc-d75bf23adc03,1.0
2,uuid:05c3a6f6-2b1b-4ee0-b0e8-fe2393440f2c,1.0
3,uuid:06938c8c-019d-42be-a04f-edbbad315098,1.0
4,uuid:0992c731-d1fb-45f2-abd2-fb3c620790f8,1.0


## Appliances
- Presence_phone_charger
- Presence_iron
- Presence_TV
- Presence_refrigerator/freezer
- Presence_radio/stereo
- Presence_DVD_player
- Presence_fan


In [29]:
MTF_HH_SectionC3 = ODEDataset("liberia/MTF_HH_SectionC3")
MTF_HH_SectionC3.from_csv(root.joinpath("MTF_HH_SectionC3.csv"))

MTF_HH_SectionC3 = MTF_HH_SectionC3.apply(common_modifiers.rename({
    "PARENT_KEY": ID_COL
}))

MTF_HH_SectionC3 = MTF_HH_SectionC3.group_by(ID_COL)

MTF_HH_SectionC3.preview()

,ID,C169_pos,C169_lbl,C169_rpt_pos,C169a,C169b,C169c1,C169c2,C169c3,C169c4,C169c5,C169c6,C169c7,C169c8,C169d,C169e,KEY,SETOFC169_rpt
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,"[C176, C203, C205]","[VCD/DVD, Radio, Regular Color TV]","[1, 2, 3]","[1.0, 1.0, 1.0]","[2.0, 2.0, 2.0]","[6.0, 6.0, 5.0]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[3.0, 2.0, 4.0]","[3.0, 2.0, 4.0]",[uuid:0006bfdb-a021-44c9-8250-016c2fb79147/Ss-...,[uuid:0006bfdb-a021-44c9-8250-016c2fb79147/Ss-...
1,uuid:002662e7-c92b-4c89-9a80-73c90a86face,[C174],[Torch/flashlight/ lantern],[1],[1.0],[6.0],[5.0],[nan],[nan],[nan],[nan],[nan],[nan],[nan],[12.0],[12.0],[uuid:002662e7-c92b-4c89-9a80-73c90a86face/Ss-...,[uuid:002662e7-c92b-4c89-9a80-73c90a86face/Ss-...
2,uuid:00498b66-fa00-424a-b332-c05cfd10c809,[C174],[Torch/flashlight/ lantern],[1],[2.0],[6.0],[5.0],[6.0],[7.0],[8.0],[nan],[nan],[nan],[nan],[12.0],[12.0],[uuid:00498b66-fa00-424a-b332-c05cfd10c809/Ss-...,[uuid:00498b66-fa00-424a-b332-c05cfd10c809/Ss-...
3,uuid:00795ed5-ef3f-4dcf-98dc-d32418fc1bdd,"[C173, C174, C176]","[Light Bulb, Other/Unknown type, Torch/flashli...","[1, 2, 3]","[1.0, 2.0, 1.0]","[2.0, 6.0, 2.0]","[5.0, 5.0, 5.0]","[6.0, 6.0, 6.0]","[7.0, nan, nan]","[8.0, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[8.0, 3.0, 4.0]","[7.0, 4.0, 5.0]",[uuid:00795ed5-ef3f-4dcf-98dc-d32418fc1bdd/Ss-...,[uuid:00795ed5-ef3f-4dcf-98dc-d32418fc1bdd/Ss-...
4,uuid:009eeb36-f8b5-4afe-b3a4-f9bcae4e4086,"[C173, C177, C206]","[Light Bulb, Other/Unknown type, Fan, Flat col...","[1, 2, 3]","[3.0, 1.0, 1.0]","[3.0, 3.0, 3.0]","[1.0, 1.0, 1.0]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[nan, nan, nan]","[3.0, 3.0, 3.0]","[3.0, 3.0, 5.0]",[uuid:009eeb36-f8b5-4afe-b3a4-f9bcae4e4086/Ss-...,[uuid:009eeb36-f8b5-4afe-b3a4-f9bcae4e4086/Ss-...


In [30]:
# - Presence_phone_charger
MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_smart_phone_charger',
                                                appliances_modifiers.presence_appliances('C169_lbl',
                                                                                         'Smartphone (internet phone) charger'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_regular_phone_charger',
                                                appliances_modifiers.presence_appliances('C169_lbl',
                                                                                         'Regular mobile phone charger'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_phone_charger', common_modifiers.multi_unify_presence(
    ['Presence_smart_phone_charger', 'Presence_regular_phone_charger']))

# - Presence_iron
MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_iron',
                                                appliances_modifiers.presence_appliances('C169_lbl', 'Iron'))

# - Presence_TV
MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_black_white_TV',
                                                appliances_modifiers.presence_appliances('C169_lbl',
                                                                                         'Black & White TV'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_regular_color_TV',
                                                appliances_modifiers.presence_appliances('C169_lbl',
                                                                                         'Regular Color TV'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_flat_color_TV',
                                                appliances_modifiers.presence_appliances('C169_lbl', 'Flat color TV'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_TV', common_modifiers.multi_unify_presence(
    ['Presence_black_white_TV', 'Presence_regular_color_TV', 'Presence_flat_color_TV']))

# - Presence_refrigerator/freezer
MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_refrigerator',
                                                appliances_modifiers.presence_appliances('C169_lbl', 'Refrigerator'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_freezer',
                                                appliances_modifiers.presence_appliances('C169_lbl', 'Freezer'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_refrigerator/freezer', common_modifiers.multi_unify_presence(
    ['Presence_refrigerator', 'Presence_freezer']))

# - Presence_radio/stereo
MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_radio',
                                                appliances_modifiers.presence_appliances('C169_lbl', 'Radio'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_stereo', appliances_modifiers.presence_appliances('C169_lbl',
                                                                                                            'Radio/CD Players/sound system'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_radio/stereo', common_modifiers.multi_unify_presence(
    ['Presence_radio', 'Presence_stereo']))

# - Presence_DVD_player
MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_DVD_player',
                                                appliances_modifiers.presence_appliances('C169_lbl', 'VCD/DVD'))
# - Presence_fan
MTF_HH_SectionC3 = MTF_HH_SectionC3.new_feature('Presence_fan',
                                                appliances_modifiers.presence_appliances('C169_lbl', 'Fan'))

MTF_HH_SectionC3 = MTF_HH_SectionC3.select([
    ID_COL, 'Presence_phone_charger', 'Presence_iron', 'Presence_TV', 'Presence_refrigerator/freezer',
    'Presence_radio/stereo', 'Presence_DVD_player', 'Presence_fan'
])

MTF_HH_SectionC3.preview()

,ID,Presence_phone_charger,Presence_iron,Presence_TV,Presence_refrigerator/freezer,Presence_radio/stereo,Presence_DVD_player,Presence_fan
0,uuid:0006bfdb-a021-44c9-8250-016c2fb79147,0,0,1,0,1,1,0
1,uuid:002662e7-c92b-4c89-9a80-73c90a86face,0,0,0,0,0,0,0
2,uuid:00498b66-fa00-424a-b332-c05cfd10c809,0,0,0,0,0,0,0
3,uuid:00795ed5-ef3f-4dcf-98dc-d32418fc1bdd,0,0,0,0,0,1,0
4,uuid:009eeb36-f8b5-4afe-b3a4-f9bcae4e4086,0,0,1,0,0,0,1


## Merge the datasets

In [31]:
Liberia = MTF_HH_Main.merge(MTF_HH_Characteristics, on=ID_COL)
Liberia = Liberia.merge(MTF_HH_SectionH, on=ID_COL)
Liberia = Liberia.merge(MTF_HH_SectionM, on=ID_COL)
Liberia = Liberia.merge(MTF_HH_SectionC3, on=ID_COL)

Liberia = Liberia.apply(common_modifiers.add_const_driver_many({
    'Years_of_HHH_in_community': np.nan,
    'Ownership_large_livestock': np.nan,
    'Ownership_small_livestock': np.nan,
    'Tariff_payment_frequency': np.nan,
    'Hours_available_electricity': np.nan
}))
Liberia.preview()

,ID,Dwelling_quality_index,Measurement_age,Monthly_expenditure,Number_of_rooms,Climate_zone_lev_1,Climate_zone_lev_2,Age_HHH,Education_level_HHH,Socio_status_HHH,...,Presence_TV,Presence_refrigerator/freezer,Presence_radio/stereo,Presence_DVD_player,Presence_fan,Years_of_HHH_in_community,Ownership_large_livestock,Ownership_small_livestock,Tariff_payment_frequency,Hours_available_electricity
0,uuid:0158227d-b2fb-4661-a791-1a4db82abaca,4.6,NaN,42290.833333,4.0,11.73975,NaN,35.0,Lower secondary education,NaN,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,uuid:06938c8c-019d-42be-a04f-edbbad315098,5.0,NaN,69136.666667,3.0,16.19575,16.734249,40.0,Doctoral or equivalent level,Own-account worker non-farm,...,1,1,1,0,1,NaN,NaN,NaN,NaN,NaN
2,uuid:0992c731-d1fb-45f2-abd2-fb3c620790f8,5.6,NaN,54068.333333,4.0,18.89050,NaN,57.0,Lower secondary education,Employee non-farm,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
3,uuid:09accf37-1bd9-473a-be76-8b3529892ee1,NaN,NaN,400092.000000,4.0,18.89050,18.890499,55.0,NaN,Worker not classifiable by status,...,0,0,1,0,0,NaN,NaN,NaN,NaN,NaN
4,uuid:0a12c98d-6d87-4de4-8087-d606bc9c915f,NaN,1.0,12638.333333,1.0,11.73975,NaN,20.0,Lower secondary education,Worker not classifiable by status,...,1,0,1,1,0,NaN,NaN,NaN,NaN,NaN


In [32]:
Liberia = Liberia.select([
                             ID_COL
                         ] + constants.DRIVERS_LIST + constants.PRESENCE_LIST)

In [33]:
Liberia.to_csv(root.joinpath("Liberia.csv"))

In [34]:
Liberia.preview()

,ID,Years_of_HHH_in_community,Dwelling_quality_index,Hours_available_electricity,Measurement_age,Monthly_expenditure,Number_of_rooms,Climate_zone_lev_1,Climate_zone_lev_2,Age_HHH,...,Ownership_large_livestock,Clean_fuel,Tariff_payment_frequency,Presence_refrigerator/freezer,Presence_iron,Presence_fan,Presence_DVD_player,Presence_radio/stereo,Presence_phone_charger,Presence_TV
0,uuid:0158227d-b2fb-4661-a791-1a4db82abaca,NaN,4.6,NaN,NaN,42290.833333,4.0,11.73975,NaN,35.0,...,NaN,No,NaN,0,0,0,0,0,0,0
1,uuid:06938c8c-019d-42be-a04f-edbbad315098,NaN,5.0,NaN,NaN,69136.666667,3.0,16.19575,16.734249,40.0,...,NaN,No,NaN,1,0,1,0,1,1,1
2,uuid:0992c731-d1fb-45f2-abd2-fb3c620790f8,NaN,5.6,NaN,NaN,54068.333333,4.0,18.89050,NaN,57.0,...,NaN,No,NaN,0,0,0,0,0,0,0
3,uuid:09accf37-1bd9-473a-be76-8b3529892ee1,NaN,NaN,NaN,NaN,400092.000000,4.0,18.89050,18.890499,55.0,...,NaN,No,NaN,0,0,0,0,1,0,0
4,uuid:0a12c98d-6d87-4de4-8087-d606bc9c915f,NaN,NaN,NaN,1.0,12638.333333,1.0,11.73975,NaN,20.0,...,NaN,No,NaN,0,0,0,1,1,0,1


In [35]:
len(Liberia)

172